In [2]:
# | default_exp packaging

In [3]:
# | export

import os
import subprocess
import sys
from configparser import ConfigParser
from pathlib import Path
from typing import List
from urllib.parse import urlparse

import yaml
from fastcore.script import call_parse
from nbdev.export import get_config

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# | export


def reqs_file_to_sep_str(pip_reqs_path: Path) -> str:
    with open(pip_reqs_path, "r") as pip_reqs_file:
        lines = pip_reqs_file.readlines()
    return reqs_lines_to_sep_str(lines)

In [6]:
# | export


def run_py_module(command, args, env=None):
    output = subprocess.run(
        [sys.executable, "-m", command, *(str(i).strip() for i in args)],
        stderr=subprocess.PIPE,
        stdout=subprocess.PIPE,
        env=env,
        universal_newlines=True,
    )

    output_code = output.returncode
    output.stdout
    err = output.stderr

    if output_code != 0:
        raise EnvironmentError(err)
    return output_code

In [37]:
# | export


def determine_dependencies(
    out_dir: Path = None, generated_pip_file_name: str = "requirements-generated.txt"
):
    try:
        import pigar
    except:
        print("Pigar dependency is not installed - not able to determine dependencies")
        return
    lib_path = get_config().path("lib_path")
    if out_dir is None:
        out_dir = lib_path.resolve().parent

    command = "pigar"
    args = ["generate", "-f", str(Path(out_dir, generated_pip_file_name))]

    run_py_module(command, args)

    return reqs_file_to_sep_str(os.path.join(out_dir, generated_pip_file_name))

In [38]:
# | export


def reqs_lines_to_sep_str(req_lines: List[str], sep: str = " "):
    return " ".join(
        [
            l.replace(" ", "").strip()
            for l in req_lines
            if not l.startswith("#") and len(l.strip()) > 0
        ]
    )

In [39]:
test_dir = Path("test").resolve()
generated_reqs_path = os.path.join(test_dir, "requirements-generated.txt")
if os.path.exists(generated_reqs_path):
    os.remove(generated_reqs_path)
assert not os.path.exists(generated_reqs_path)
determine_dependencies(out_dir=test_dir)
assert os.path.exists(generated_reqs_path)
os.remove(generated_reqs_path)

# Requirement.txt Manipulation

> Read pip requirements file and convert to a structure that can be used to transform that output to a different format.

For more information see here:

https://www.python.org/dev/peps/pep-0440/#version-specifiers

In [40]:
test_lines = (
    "fastcore == 1.3.19",
    "\n",
    "#",
    "nbformat >= 5.0.8",
    "# scidev/nb_lint.py: 10,11,12",
    "nbqa ~= 0.5.6",
    "nbqa <=0.5.6",
)

In [41]:
assert (
    "fastcore==1.3.19 nbformat>=5.0.8 nbqa~=0.5.6 "
    "nbqa<=0.5.6" == reqs_lines_to_sep_str(test_lines)
)

In [42]:
determine_dependencies(out_dir=test_dir)
reqs_str = reqs_file_to_sep_str(generated_reqs_path)
os.remove(generated_reqs_path)

In [43]:
# | export


def update_requirements(
    project_dir: Path = None, output_filename: str = "settings.ini"
):
    if project_dir is None:
        lib_path = get_config().path("lib_path")
        project_dir = lib_path.resolve().parent

    config = ConfigParser(delimiters=["="])
    settings_path = os.path.join(project_dir, "settings.ini")
    config.read(settings_path)

    os.path.join(project_dir, "requirements-generated.txt")
    reqs_str = determine_dependencies(out_dir=project_dir)

    out_path = os.path.join(project_dir, output_filename)
    config.set("DEFAULT", "requirements", reqs_str)

    with open(out_path, "w") as configfile:
        config.write(configfile)

In [44]:
determine_dependencies(out_dir=test_dir)
update_requirements(test_dir)

In [45]:
config = ConfigParser(delimiters=["="])
test_config_file = os.path.join(test_dir, "settings.ini")
config.read(test_config_file)
assert "nbdev" in config.get("DEFAULT", "requirements")

In [46]:
required_keys = (
    "lib_name",
    "description",
    "version",
    "custom_sidebar",
    "license",
    "status",
    "console_scripts",
    "nbs_path",
    "lib_path",
    "title",
    "tst_flags",
)

In [47]:
assert all([config.get("DEFAULT", k) is not None for k in required_keys])

# Create conda build file

In [48]:
# | export


def create_conda_meta_file(project_dir: Path = None, out_file: str = "meta.yaml"):
    if project_dir is None:
        lib_path = get_config().path("lib_path")
        project_dir = lib_path.resolve().parent

    meta_data = {
        "package": {
            "name": get_config().get("lib_name"),
            "version": get_config().get("version"),
        },
        "source": {"path": str(get_config().path("lib_path").resolve().parent)},
        "requirements": {
            "host": ["pip", "python", "setuptools"],
            "run": determine_dependencies(out_dir=project_dir).split(" "),
        },
    }
    with open(os.path.join(project_dir, out_file), "w") as conda_build_file:
        yaml.dump(meta_data, conda_build_file)

In [49]:
create_conda_meta_file(Path("test"))

# Update All Project Requirements

In [50]:
# | export


@call_parse
def sciflow_update_reqs():
    create_conda_meta_file()
    update_requirements()
    print("Updated library requirements for conda & nbdev")

This code does not work as expected the pigar library seems to have limitations in practice for determining all dependencies making this route not viable just yet.

# Prepare Artifactory Environment

> This code should be in projects not here.

In [51]:
# | export


def delete_multiple_element(list_object, indices):
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx)

In [52]:
# | export


def read_deploy_vars():
    with open(os.path.join(Path.home(), ".condarc"), "r") as conda_rc_file:
        conda_rc = yaml.load(conda_rc_file, Loader=yaml.FullLoader)
        conda_url = conda_rc["channels"][0]
    deployment = {
        "conda_url": conda_url,
        "artifactory_user": urlparse(conda_url).netloc.split(":")[0],
        "artifactory_token": urlparse(conda_url).netloc.split(":")[1].split("@")[0],
        "artifactory_url": urlparse(conda_url).netloc.split(":")[1].split("@")[1],
        "artifactory_conda_channel": "conda-local",
        "lib_name": get_config().get("lib_name"),
        "version": get_config().get("version"),
        "build_number": 0,
    }
    return deployment

In [53]:
deploy_vars = read_deploy_vars()
assert deploy_vars["lib_name"] == "sciflow"

In [54]:
# | export


def write_art_conda_envs_to_file():
    dep_vars = read_deploy_vars()

    with open(os.path.join(Path.home(), ".profile"), "r") as profile_file:
        existing_lines = profile_file.readlines()
        to_remove = []
        for i, line in enumerate(existing_lines):
            if (
                line.strip().startswith("export ARTIFACTORY_")
                or line.strip().startswith("export LIB_NAME")
                or line.strip().startswith("export VERSION")
                or line.strip().startswith("export BUILD_NUMBER")
            ):
                to_remove.append(i)
            if not line.endswith("\n"):
                existing_lines[i] = line + "\n"
        delete_multiple_element(existing_lines, to_remove)

    with open(os.path.join(Path.home(), ".profile"), "w") as profile_file:
        new_lines = [
            "export ARTIFACTORY_USER={artifactory_user}\n".format(**dep_vars),
            "export ARTIFACTORY_PASSWORD={artifactory_token}\n".format(**dep_vars),
            "export ARTIFACTORY_URL={artifactory_url}\n".format(**dep_vars),
            "export ARTIFACTORY_CONDA_CHANNEL={artifactory_conda_channel}\n".format(
                **dep_vars
            ),
            "export LIB_NAME={lib_name}\n".format(**dep_vars),
            "export VERSION={version}\n".format(**dep_vars),
            "export BUILD_NUMBER={build_number}\n".format(**dep_vars),
        ]
        existing_lines.extend(new_lines)
        profile_file.writelines(existing_lines)
    return existing_lines

In [55]:
lines = write_art_conda_envs_to_file()

In [56]:
assert len(lines) >= 7

In [57]:
# | export


@call_parse
def sciflow_prepare():
    dep_vars = read_deploy_vars()

    for dep_key in dep_vars.keys():
        os.environ[dep_key.upper()] = str(dep_vars[dep_key])

In [58]:
if "ARTIFACTORY_CONDA_CHANNEL" in os.environ:
    del os.environ["ARTIFACTORY_CONDA_CHANNEL"]

In [59]:
sciflow_prepare()

In [60]:
assert "conda-local" == os.environ["ARTIFACTORY_CONDA_CHANNEL"]

In [61]:
os.environ

environ{'MAMBA_USER_GID': '57439',
        'REGION_NAME': 'eu-west-1',
        'HOSTNAME': 'sagemaker-distribution-ml-m5-large-91cc60800b4f728f1fcc6d84329a',
        'ENV_NAME': 'base',
        'HOME': '/home/sagemaker-user',
        'MAMBA_USER': 'sagemaker-user',
        'MAMBA_USER_ID': '57439',
        'AWS_CONTAINER_CREDENTIALS_RELATIVE_URI': '/_sagemaker-instance-credentials/a0a5b692448469dacfecfb048eeb459258b9a2da6a0b1198bd05313b52eff277',
        'PYTHONNOUSERSITE': '0',
        'AWS_DEFAULT_REGION': 'eu-west-1',
        'PATH': '/opt/conda/bin:/opt/conda/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tmp/miniconda3/condabin:/tmp/anaconda3/condabin:/tmp/miniconda2/condabin:/tmp/anaconda2/condabin:/tmp/mambaforge/condabin',
        'MAMBA_ROOT_PREFIX': '/opt/conda',
        'LANG': 'C.UTF-8',
        'AWS_ACCOUNT_ID': '823878111748',
        'SHELL': '/bin/bash',
        'AWS_REGION': 'eu-west-1',
        'AWS_INTERNAL_IMAGE_OWNER': 'Studio',
        'MAM